# <center> Introduction to Reinforcement Learning</center>

# Setup

Install required dependencies (first time might take a while)

In [ ]:
running_locally = True # set this to False if you are running on colab

In [ ]:
import sys, os

# running in local environment or on Binder
if running_locally:
    !pip install gym matplotlib
    sys.path.append('Support')
    sys.path.insert(0, os.path.abspath('Support'))

# running in Google Colab
else:
    !pip install gym
    from google.colab import drive
    drive.mount('/content/gdrive',force_remount=True)
    # replace these Google Drive paths with your own (images in markdown cells may not load correctly in colab)
    sys.path.append('/content/gdrive/MyDrive/Colab/ECE4078_2021_Practicals/Week07/Support')
    sys.path.insert(1, os.path.abspath('/content/gdrive/MyDrive/Colab/ECE4078_2021_Practicals/Week07/Support'))
    !ls '/content/gdrive/MyDrive/Colab/ECE4078_2021_Practicals/Week07/Support'

Import dependencies

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import pickle

import sys
import os

from gym_simple_gridworlds.envs.grid_env import GridEnv
from gym_simple_gridworlds.envs.grid_2dplot import *
from gym_simple_gridworlds.helper import *

Set up a wide cell view, comment out this cell if you prefer narrow view

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Activity 1. Elements of an MDP (Grid World Example)

![Grid World](Support/images/GridWorldExample.png)

- The states $s \in \mathcal{S}$ correspond to locations in the grid. Each location has also a cell index associated to it, e.g., cell index 4 is associated to location (row=1,col=0)
- The robot can move up, down, left, or right. Actions correpond to unit increments or decrements in the specified direction.
    - Up : (-1,0)
    - Down: (1,0)
    - Left: (0,-1)
    - Right: (0, 1)
- Each action is represented by a number. Action (Up/North) is represented by 0, (Down/South) by 1, (Left/West) by 2 and, finally, (Right/East) by 3. No actions are available at a terminal state

We have defined the class ``GridEnv`` to represent our Grid World MDP. *Hover your cursor on "GridEnv" and press shift-Tab, this shows more information on how the grid world instance is defined.*

## Create Environment and Explore its Attributes

The noise parameter corresponds to the probability of a change of direction when an action is taken (e.g., going left/right when agent decides to move up/down)

In [ ]:
# Create a Grid World instance
grid_world = GridEnv(gamma=0.9, noise=0.2, living_reward=-0.04)

### State and Action Spaces

Let's take a look at the state and action spaces of our environment

In [ ]:
# State (or observation) space
print(grid_world.observation_space)
print(grid_world.get_states())
print()

# Action space
print(grid_world.action_space)
print(grid_world.get_actions())

### Transition Function

Let's take a look at the current state transition function. Some things to keep in mind regarding the transition function:

![Noisy State Transition](Support/images/mdp_transitions.png)

1. Given that $\mathcal{T}: \mathcal{S} \times \mathcal{A} \times \mathcal{S} \rightarrow \mathbb{R}$, the ``state_transitions`` attribute of the class ``GridEnv`` corresponds to a 3-Dimensional numpy array of size $11\times4\times11$.
2. With a noise attribute set to 0.2, at state 9, if the agent chooses to move up, it will end up at:
    - state 5 with $80\%$ probability,
    - state 8 with $10\%$ probability, or
    - state 10 with $10\%$ probability

In [ ]:
# at state 5 the agent takes action 0 (going up)
print(grid_world.state_transitions[9,0])

### Living Reward and Reward Function

Let's now take a quick look at the living reward (i.e., running cost) and reward function $\mathcal{R}: \mathcal{S} \times \mathcal{A} \rightarrow \mathbb{R}$.

1. Living reward corresponds to the attribute ``living_rewards`` of the class ``GridEnv`` and is represented as an 1-Dimensional numpy array
2. The reward function corresponds to the attribute ``rewards`` of the class ``GridEnv`` and is also represented as a 2-Dimensional numpy array of size $11\times4$

**Quiz 2: what is the expected reward at state 2 if the agent chooses to move right?**

In [ ]:
# Living rewards
print("Living rewards for all states:\n{}\n".format(grid_world.immediate_rewards))

# Reward function, i.e., expected reward for taking action a at state s
print("Reward function for all state-action pairs:\n{}\n".format(grid_world.rewards))
print("The expected reward at state 5 if agent chooses to move right is: {}".format(grid_world.rewards[5,3]))

# To do (Quiz 2): what is the expected reward at state 2 if the agent chooses to move right?

# Activity 2. Policy in an MDP

Let's see the path and total reward of an agent moving on our grid world according to the following policy $\pi$

![Example Policy](Support/images/example_policy.png)

In [ ]:
# We represent this policy as a 2-Dimensional numpy array
policy_matrix = np.array([[3,      3,  3,  -1],
                          [0, np.NaN,  0,  -1],
                          [0,      2,  0,   2]])

In [ ]:
print(grid_world.grid)

Let's now apply this policy and observe the agent's behavior (blue dot in the video shown below). Try this a few times.

*If you got the error "Requested MovieWriter (ffmpeg) not available" when running the notebook locally, run ``conda install -c conda-forge ffmpeg`` in your anaconda command prompt and restart the kernal*

In [ ]:
# Create a Grid World instance
grid_world = GridEnv(gamma=0.9, noise=0.2, living_reward=-0.04)
s_x, s_y = get_state_to_plot(grid_world)

# We can visualize our grid world using the render() function
fig, ax = grid_world.render()
agent, = ax.plot([], [], 'o', color='b', linewidth=6)
reward_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)

done = False
cumulative_reward = 0
cur_state = grid_world.cur_state
path_to_plot = []

while not done:
    _, cur_reward, done, _ = grid_world.step(int(policy_matrix[cur_state[0], cur_state[1]]))
    cur_state = grid_world.cur_state
    n_x, n_y = get_state_to_plot(grid_world)
    cumulative_reward += cur_reward
    path_to_plot.append([cumulative_reward, n_x, n_y])

def init():
    agent.set_data(s_x + 0.5, s_y + 0.5)
    reward_text.set_text('')
    return agent, reward_text

def animate(i):
    if i < len(path_to_plot):
        r, n_x, n_y = path_to_plot[i]
        agent.set_data(n_x + 0.5, n_y + 0.5)
        reward_text.set_text('Cumulative reward: %.2f' % r)
    return agent, reward_text

ani = animation.FuncAnimation(fig, animate, frames=len(path_to_plot), blit=False, interval=500, init_func=init,
                              repeat=False)

html = HTML(ani.to_html5_video())
display(html)
plt.close()